In [ ]:
'''
So take top 10 companies by sector instead of all 8800 thousand
(DONE)

:
(pre-processing) do on all data before 2020 Do all reproducing from lecturers after cleaning check for correlations (Bar chart all this):
            Simple linear regression (DONE)
            Multi linear regressions (DONE)
            logistic Regression (Sergei Did this so copy sergei code) (DONE)
            Missing values see if there is any missing values still (DONE)
            Outlier detection (3 std deviation) (DONE)
            Data Transformation (We already did data normalisation), 
                                    we need to do discretisation (DONE)
            Feature Enginnering we need to remove inflation adjsuted clsoign prices (already done)

When you split the data make sure you split from 2008 to 2020 is training data, then 2020 - now is training data
top 10 comapnies per sector right now
(DONE)
'''

In [ ]:
'''
Energy,XOM,CVX,COP,WMB,EPD,MPC,ET,MPLX,PSX,VLO
Materials,LIN,SCCO,SHW,ECL,CRH,FCX,APD,CTVA,AU,VMC
Industrial,GE,CAT,RTX,GEV,BA,UNP,ETN,DE,HON,LMT
Consumer discretionary,AMZN,TSLA,BABA,HD,TM,MCD,PDD,TJX,BKNG,LOW
Consumer staples,WMT,COST,PG,KO,PM,PEP,UL,BTI,BUD,MO
Health care,LLy,JNJ,ABBV,UNH,MRK,ABT,ISRG,AMGN,DHR,GILD
Financials,JPM,V,BAC,GS MS,WFC,C,AXP,RY
Information technology,AAPL,MSFT,GOOG,AVGO,TSMC,ORCL,PLTR,AMD,CSCO,MU
Communication,GOOG,META,NFLX,TMUS,DIS,APP,T,VZ,SPOT,CMCSA
Utilities,CEG,SO,DUK,AEP,VST,SRE,D,XEL,EXC,ETR
Real estate,PLD,AMT,EQIX,SPG,DLR,O,PSA,CBRE,CCI,VTR
'''

In [ ]:
# Cleaning pipeline

# -*- coding: utf-8 -*-
"""
Created on Thu Nov 20 17:48:05 2025

@author: Enrico
"""

import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf
from pandas_datareader.data import DataReader


def build_market_info():
    raw = [
        ("BITCOIN", "Digital Currencies"),
        ("GOLD", "Commodities"),
        ("SILVER", "Commodities"),
        ("OIL", "Commodities"),
        ("VIX", "Volatility Index"),
        ("FTSE100", "Equities"),
        ("DAX", "Equities"),
        ("CAC40", "Equities"),
        ("NIKKEI", "Equities"),
        ("SP500", "Equities"),
        ("NASDAQ", "Equities"),
        ("REIT_USA", "Real Estate Investment Trusts (REIT)"),
        ("REIT_EUROPE", "Real Estate Investment Trusts (REIT)"),
        ("REIT_ASIA", "Real Estate Investment Trusts (REIT)")
    ]
    df = pd.DataFrame(raw, columns=["symbol (acronym)", "market_group"])
    df["Currency"] = "USD"
    df.loc[df["symbol (acronym)"] == "DAX", "Currency"] = "EUR"
    df.loc[df["symbol (acronym)"] == "CAC40", "Currency"] = "EUR"
    return df

market_info = build_market_info()

def normalize_columns(df):
    """Normalize column names: lowercase, strip spaces, replace non-alphanum with underscores."""
    df = df.copy()
    df.columns = (
        df.columns.str.strip()
                  .str.lower()
                  .str.replace(r'\s+', '_', regex=True)
                  .str.replace(r'[^\w]', '', regex=True)
    )
    return df

def _dedupe_ohlcv(df):
    """
    Remove duplicate dates in OHLCV data, keeping the last occurrence.
    Assumes 'date' column exists.
    """
    if 'date' not in df.columns:
        raise ValueError("DataFrame must have a 'date' column")
    df = df.sort_values('date')  # ensure chronological order
    df = df.drop_duplicates(subset='date', keep='last')
    df = df.set_index('date')
    return df


def get_exchange_rate(currency, start_date, end_date):
    """Get daily exchange rate to USD."""
    if currency.upper() == 'USD':
        df = pd.DataFrame({
            'Date': pd.date_range(start=start_date, end=end_date),
            'Rate_to_USD': 1.0
        })
        return df
    fx_map = {
        'EUR': ('EURUSD=X', 1),
        'GBP': ('GBPUSD=X', 1),
        'AUD': ('AUDUSD=X', 1),
        'NZD': ('NZDUSD=X', 1),
        'CAD': ('USDCAD=X', -1),
        'JPY': ('USDJPY=X', -1),
        'CHF': ('USDCHF=X', -1),
        'CNY': ('USDCNY=X', -1),
        'HKD': ('USDHKD=X', -1),
        'SEK': ('USDSEK=X', -1),
        'NOK': ('USDNOK=X', -1),
        'SGD': ('USDSGD=X', -1)
    }

    cur = currency.upper()
    if cur in fx_map:
        symbol, invert = fx_map[cur]
    else:
        symbol, invert = (f"{cur}USD=X", 1)  # fallback

    data = yf.download(symbol, start=start_date, end=end_date)[['Close']].reset_index()
    data = data.rename(columns={'Close': 'Rate_to_USD'})
    if invert == -1:
        data['Rate_to_USD'] = 1 / data['Rate_to_USD']
    return data[['Date','Rate_to_USD']]

def convert_to_usd(df, currency, exchange_rates):
    """
    Merge df with exchange rates and compute Value_USD.
    Ensures no MultiIndex issues.
    """
    df = df.copy()

    # Reset index if it's a MultiIndex
    if isinstance(df.index, pd.MultiIndex):
        df = df.reset_index()

    # Ensure 'Date' column exists for merge
    if 'Date' not in df.columns:
        df['Date'] = df.index

    df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize(None)
    exchange_rates['Date'] = pd.to_datetime(exchange_rates['Date']).dt.tz_localize(None)

    merged = pd.merge(df, exchange_rates, on='Date', how='left')

    # Use 'close' if exists, otherwise fallback to first numeric column
    if 'close' in merged.columns:
        merged['Value_USD'] = merged['close'] * merged['Rate_to_USD']
    else:
        first_numeric = merged.select_dtypes(include='number').columns[0]
        merged['Value_USD'] = merged[first_numeric] * merged['Rate_to_USD']

    merged = merged.set_index('Date')
    return merged


def get_inflation_index(start_date, end_date):
    """Fetch daily interpolated US CPI from FRED."""
    try:
        cpi = DataReader('CPIAUCNS', 'fred', start_date - pd.DateOffset(days=31), end_date)
    except Exception as e:
        print("FRED CPI fetch failed:", e)
        return pd.DataFrame({'Date': pd.date_range(start=start_date, end=end_date), 'CPI_USD': np.nan})

    cpi = cpi.reset_index().rename(columns={'DATE':'Date','CPIAUCNS':'CPI_USD'})
    cpi['Date'] = pd.to_datetime(cpi['Date'])

    daily_index = pd.DataFrame({'Date': pd.date_range(start=start_date, end=end_date)})
    daily_index = pd.merge_asof(daily_index.sort_values('Date'),
                                cpi.sort_values('Date'),
                                on='Date',
                                direction='backward')
    daily_index['CPI_USD'] = daily_index['CPI_USD'].ffill().bfill().interpolate(method='linear')
    return daily_index

def adjust_for_inflation(df, inflation_index):
    df = df.copy()
    df['Date'] = df.index
    inflation_index['Date'] = pd.to_datetime(inflation_index['Date']).dt.tz_localize(None)
    merged = df.merge(inflation_index, on='Date', how='left')
    latest_cpi = inflation_index['CPI_USD'].iloc[-1]
    merged['real_close'] = merged['Value_USD'] * (latest_cpi / merged['CPI_USD'])
    merged = merged.set_index('Date')
    return merged

def euclidean_vector(a, b):
    a = np.array(a, dtype=float)
    b = np.array(b, dtype=float)
    if a.shape != b.shape:
        raise ValueError("Vectors must be same length")
    return float(np.sqrt(np.sum((a - b) ** 2)))

def chi_square_test(observed, expected):
    observed = np.array(observed, dtype=float)
    expected = np.array(expected, dtype=float)
    if observed.shape != expected.shape:
        raise ValueError("Length mismatch")
    if np.any(expected == 0):
        raise ValueError("Zero expected freq not allowed")
    chi_sq = np.sum((observed - expected) ** 2 / expected)
    return chi_sq

def process_asset_file(filepath, asset_name, market_info):
    """
    Process a single asset CSV file:
    - Clean columns and normalize names
    - Ensure date column exists
    - Deduplicate OHLCV data
    - Convert to USD if necessary
    - Adjust for inflation
    """
    # Load CSV
    df = pd.read_csv(filepath, on_bad_lines="skip", low_memory=False)

    # Normalize columns
    df = normalize_columns(df)

    # Find date-like column
    date_col = next((c for c in df.columns if "date" in c or "time" in c), None)
    if date_col is None:
        raise ValueError(f"No date-like column found in {filepath}")

    # Rename to 'date'
    if date_col != "date":
        df = df.rename(columns={date_col: "date"})

    # Convert date to datetime
    df['date'] = pd.to_datetime(df['date'], errors="coerce")
    df = df.dropna(subset=['date'])
    df = df.sort_values('date')

    # Deduplicate OHLCV
    df = _dedupe_ohlcv(df)

    # Skip if empty
    if df.empty:
        print(f"Skipping {asset_name}: no valid dates after cleaning")
        return pd.DataFrame()

    # Get currency
    row = market_info.loc[market_info['symbol (acronym)'] == asset_name]
    currency = row['Currency'].iloc[0] if len(row) else 'USD'

    # Convert to USD if needed
    if currency != "USD":
        fx = get_exchange_rate(currency, df.index.min(), df.index.max())
        df = convert_to_usd(df, currency, fx)
    else:
        df['Value_USD'] = df.get('close', np.nan)

    # Inflation adjustment
    inflation_index = get_inflation_index(df.index.min(), df.index.max())
    df = adjust_for_inflation(df, inflation_index)

    return df



def looks_like_returns_matrix(df):
    if not isinstance(df.index, pd.DatetimeIndex):
        try: df.index = pd.to_datetime(df.index)
        except: return False
    if not df.apply(lambda s: pd.api.types.is_numeric_dtype(s)).all():
        return False
    if not df.columns.str.contains(r"^\d").any():
        return False
    return True

def clean_edhec_returns(df, source):
    df = df.copy()
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

def clean_names_and_dtypes(df, source, symbol_hint=None):
    df = normalize_columns(df)
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
        df = df.dropna(subset=["date"])
    return df

def clean_any(df, source, symbol_hint=None):
    if looks_like_returns_matrix(df.rename(columns=str.lower)):
        return clean_edhec_returns(df, source)
    return clean_names_and_dtypes(df, source, symbol_hint)

def calendar_align(df):
    df = df.copy()

    # force datetime index
    df.index = pd.to_datetime(df.index, errors="coerce")
    df = df[~df.index.isna()]

    # localize/convert to UTC
    try:
        df.index = df.index.tz_localize("UTC")
    except TypeError:
        df.index = df.index.tz_convert("UTC")

    return df

def process_folder(path, market_info):
    results = {}
    for file in os.listdir(path):
        if not file.lower().endswith(".csv"): continue
        symbol = os.path.splitext(file)[0].upper()
        df = process_asset_file(os.path.join(path, file), symbol, market_info)
        df = calendar_align(df)
        results[symbol] = df
    return results

def summarize_data(df, symbol=None):
    summary = pd.DataFrame({
        "first_date": [df.index.min()],
        "last_date": [df.index.max()],
        "num_rows": [len(df)],
        "num_missing": [df.isna().sum().sum()],
        "columns": [", ".join(df.columns)]
    })
    if symbol:
        print(f"Summary for {symbol}:")
    print(summary)
    return summary

def plot_time_series(df, column="close", symbol=None):
    # Plot original vs real_close
    if "close" in df.columns and "real_close" in df.columns:
        plt.figure(figsize=(10, 4))
        plt.plot(df.index, df["close"], label="Original Close / AdjClose", alpha=0.8)
        plt.plot(df.index, df["real_close"], label="Inflation Adjusted Close", alpha=0.8)
        plt.title(f"{symbol} Close vs Inflation-Adjusted Close")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
    else:
        print(f"{symbol}: missing 'close' or 'real_close', cannot plot comparison")

def process_folder_using_dictionary(path, market_info, dictionary_csv):

    # Read dictionary
    dict_df = pd.read_csv(dictionary_csv, header=None)

    # Extract all tickers (columns 1..end)
    all_symbols = set()

    for _, row in dict_df.iterrows():
        # skip the first entry (category)
        symbols = row.dropna().tolist()[1:]
        symbols = [str(s).strip().upper() for s in symbols]
        all_symbols.update(symbols)

    # Convert symbols → filenames
    allowed_files = {f"{symbol}.csv" for symbol in all_symbols}

    print("Allowed files:", allowed_files)

    results = {}

    for file in os.listdir(path):
        if file not in allowed_files:
            continue  # skip files not in dictionary

        symbol = os.path.splitext(file)[0].upper()
        full_path = os.path.join(path, file)

        print(f"Processing {symbol} ...")

        df = process_asset_file(full_path, symbol, market_info)
        if df.empty:
            print(f"Skipping {symbol}: empty after cleaning (not saving).")
            continue
        df = calendar_align(df)
        results[symbol] = df

    return results

folder_path = os.path.expanduser("~/csc1171/data/raw/SP500_ETF_FX_Crypto_Daily")

dictionary_path = os.path.expanduser("~/csc1171/notebooks/Dominik/company_dictionary.csv")


cleaned_data = process_folder_using_dictionary(
    folder_path,
    market_info,
    dictionary_path
)


for symbol, df in cleaned_data.items():
    print(symbol, df.head())

for symbol, df in cleaned_data.items():
    summarize_data(df, symbol)
    plot_time_series(df, "real_close", symbol)


output_folder = os.path.expanduser("~/csc1171/data/clean/SP500_ETF_FX_Crypto_Daily")

os.makedirs(output_folder, exist_ok=True)  # create folder if it doesn't exist

for symbol, df in cleaned_data.items():

    full_path = os.path.join(output_folder, f"{symbol}_cleaned_full.csv")
    df.to_csv(full_path)
    
    df_2008_2020 = df.loc[(df.index >= "2008-01-01") & (df.index <= "2020-12-31")]
    path_2008_2020 = os.path.join(output_folder, f"{symbol}_cleaned_2008_2020.csv")
    df_2008_2020.to_csv(path_2008_2020)

    df_2020_onward = df.loc[df.index >= "2020-01-01"]
    path_2020_onward = os.path.join(output_folder, f"{symbol}_cleaned_2020_onward.csv")
    df_2020_onward.to_csv(path_2020_onward)

    print(f"Saved 3 versions for {symbol}")


for symbol, df in cleaned_data.items():
    # Save each cleaned DataFrame as CSV
    output_path = os.path.join(output_folder, f"{symbol}_cleaned.csv")
    df.to_csv(output_path)
    print(f"Saved cleaned data for {symbol} to {output_path}")


In [ ]:
# Simple Linear Regression: pure secular drift in real terms
import glob, pandas as pd, numpy as np, statsmodels.api as sm

CLEAN_DIR = "/home/ug/orlovsd2/csc1171/data/clean/SP500_ETF_FX_Crypto_Daily"
files = glob.glob(f"{CLEAN_DIR}/*_cleaned_2008_2020.csv")


def load_clean_csv(path):
    try:
        df = pd.read_csv(path, parse_dates=["Date"]); df = df.set_index("Date")
    except ValueError:
        df = pd.read_csv(path, index_col=0, parse_dates=[0])
    df.index = pd.to_datetime(df.index).tz_localize(None)
    return df

y_col = "real_close"
results = []

for path in files:
    df = load_clean_csv(path)
    if y_col not in df.columns:
        continue
    dfi = df[[y_col]].dropna().copy()
    if dfi.empty:
        continue

    dfi["t"] = (dfi.index - dfi.index.min()).days
    X = sm.add_constant(dfi["t"])
    y = dfi[y_col].astype(float)

    res = sm.OLS(y, X).fit()
    ticker = path.split("/")[-1].replace("_cleaned.csv", "")

    results.append({
        "ticker": ticker,
        "beta_intercept": res.params.get("const", np.nan),
        "beta_slope": res.params.get("t", np.nan),
        "p_value_slope": res.pvalues.get("t", np.nan),
        "R2": res.rsquared,
        "Adj_R2": res.rsquared_adj,
        "AIC": res.aic,
        "BIC": res.bic,
        "n": int(res.nobs)
    })

pd.set_option("display.max_rows", None)

reg_table = pd.DataFrame(results).sort_values("p_value_slope")
reg_table



In [ ]:
# MLR: drift net of same-day price levels/volume.

'''
How much did each stock’s price linearly trend upward or downward between 2008–2020?
'''


CLEAN_DIR = "/home/ug/orlovsd2/csc1171/data/clean/SP500_ETF_FX_Crypto_Daily"
files = glob.glob(f"{CLEAN_DIR}/*_cleaned_2008_2020.csv")


def load_clean_csv(path):
    try:
        df = pd.read_csv(path, parse_dates=["Date"]); df = df.set_index("Date")
    except ValueError:
        df = pd.read_csv(path, index_col=0, parse_dates=[0])
    df.index = pd.to_datetime(df.index).tz_localize(None)
    return df

y_col = "real_close"

candidate_predictors = ["volume"]


results = []

for path in files:
    df = load_clean_csv(path)
    if y_col not in df.columns:
        continue
    # build X from whatever predictors exist in this file
    preds_use = [c for c in candidate_predictors if c in df.columns and c != y_col]
    if len(preds_use) < 1:
        continue
    
    dfi = df[[y_col] + preds_use].dropna().copy()
    dfi["t"] = (dfi.index - dfi.index.min()).days
    preds_use = preds_use + ["t"]

    if dfi.empty:
        continue
    
    X = sm.add_constant(dfi[preds_use].astype(float))
    y = dfi[y_col].astype(float)
    res = sm.OLS(y, X).fit()
    coef_tbl = pd.DataFrame({
        "term": res.params.index,
        "estimate": res.params.values,
        "std_error": res.bse.values,
        "t_value": res.tvalues.values,
        "p_value": res.pvalues.values
    })



    ticker = os.path.basename(path).replace("_cleaned_2008_2020.csv", "")

    results.append({
        "ticker": ticker,
        "beta_intercept": res.params.get("const", np.nan),
        "beta_slope": res.params.get("t", np.nan),
        "p_value_slope": res.pvalues.get("t", np.nan),
        "R2": res.rsquared,
        "Adj_R2": res.rsquared_adj,
        "AIC": res.aic,
        "BIC": res.bic,
        "n": int(res.nobs)
    })

pd.set_option("display.max_rows", None)

reg_table = pd.DataFrame(results).sort_values("p_value_slope")
reg_table



In [ ]:
# logistic Regression (Sergei Did this so copy sergei code)

import os, glob
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tools.sm_exceptions import PerfectSeparationError

CLEAN_DIR = "/home/ug/orlovsd2/csc1171/data/clean/SP500_ETF_FX_Crypto_Daily"
files = glob.glob(f"{CLEAN_DIR}/*_cleaned_2008_2020.csv")

def load_clean_csv(path):
    try:
        df = pd.read_csv(path, parse_dates=["Date"]); df = df.set_index("Date")
    except ValueError:
        df = pd.read_csv(path, index_col=0, parse_dates=[0])
    df.index = pd.to_datetime(df.index).tz_localize(None)
    return df

def make_calendar_features(df):
    """Create the calendar columns used in your snippet."""
    idx = df.index
    out = pd.DataFrame(index=idx)
    out["DayOfYear"]   = idx.dayofyear.astype(int)
    out["IsMonthStart"]= idx.is_month_start.astype(int)
    out["IsJanuary"]   = (idx.month == 1).astype(int)
    out["IsWeekend"]   = (idx.weekday >= 5).astype(int)  # you can ignore this if not needed
    return out

def safe_pct_change(s):
    r = s.pct_change()
    return r.replace([np.inf, -np.inf], np.nan)

results = []

for path in files:
    df = load_clean_csv(path)
    if "real_close" not in df.columns:
        continue  # nothing to model

    # --- target: daily return sign ---
    df["Return"] = safe_pct_change(df["real_close"])
    feats = make_calendar_features(df)

    # keep exactly your chosen predictors
    X_logit = feats[["DayOfYear", "IsMonthStart", "IsJanuary"]].copy()
    y = (df["Return"] > 0).astype(int)

    # align / drop NA
    dfi = pd.concat([X_logit, y.rename("Return_Positive")], axis=1).dropna()
    if dfi.empty or dfi["Return_Positive"].nunique() < 2:
        # need both classes to fit logit
        continue

    X = sm.add_constant(dfi[["DayOfYear", "IsMonthStart", "IsJanuary"]])
    yb = dfi["Return_Positive"]

    try:
        # Fit Logit (same as your code)
        logit_model = sm.Logit(yb, X).fit(disp=False)

        params = logit_model.params
        pvals  = logit_model.pvalues

        # predictions for a quick accuracy metric (optional)
        phat = logit_model.predict(X)
        pred = (phat >= 0.5).astype(int)
        acc = (pred == yb).mean()

        ticker = os.path.basename(path).replace("_cleaned_2008_2020.csv", "")

        results.append({
            "ticker": ticker,
            "coef_const":        params.get("const", np.nan),
            "coef_DayOfYear":    params.get("DayOfYear", np.nan),
            "coef_IsMonthStart": params.get("IsMonthStart", np.nan),
            "coef_IsJanuary":    params.get("IsJanuary", np.nan),
            "p_const":           pvals.get("const", np.nan),
            "p_DayOfYear":       pvals.get("DayOfYear", np.nan),
            "p_IsMonthStart":    pvals.get("IsMonthStart", np.nan),
            "p_IsJanuary":       pvals.get("IsJanuary", np.nan),
            "pseudoR2_McFadden": logit_model.prsquared,
            "AIC": logit_model.aic,
            "BIC": logit_model.bic,
            "n": int(logit_model.nobs),
            "accuracy@0.5": float(acc)
        })

    except PerfectSeparationError:
        print(f"SKIP {os.path.basename(path)}: PerfectSeparationError")
        continue
    except Exception as e:
        print(f"SKIP {os.path.basename(path)}: {type(e).__name__}: {e}")
        continue


pd.set_option("display.max_rows", None)
logit_table = pd.DataFrame(results)
if logit_table.empty:
    print("No models were fitted (results is empty) — likely because all rows were skipped.")
else:
    logit_table = logit_table.sort_values("p_DayOfYear")
logit_table



In [ ]:
# Missing values see if there is any missing values still

# === Missingness check across all cleaned 2008–2020 files (lecture-style) ===
import os, glob
import numpy as np
import pandas as pd

CLEAN_DIR = "/home/ug/orlovsd2/csc1171/data/clean/SP500_ETF_FX_Crypto_Daily"
files = glob.glob(f"{CLEAN_DIR}/*_cleaned_2008_2020.csv")

def load_clean_csv(path):
    try:
        df = pd.read_csv(path, parse_dates=["Date"]); df = df.set_index("Date")
    except ValueError:
        df = pd.read_csv(path, index_col=0, parse_dates=[0])
    # keep it naive as in your earlier cells
    df.index = pd.to_datetime(df.index).tz_localize(None)
    return df

per_file_rows = []
per_file_missing = []     # long form: (ticker, column, n_missing, pct_missing)
per_file_anyrow = []      # rows with ANY NaN
key_cols = ["real_close", "close", "adj_close", "open", "high", "low", "volume", "Value_USD", "CPI_USD"]

for path in files:
    df = load_clean_csv(path)
    ticker = os.path.basename(path).replace("_cleaned_2008_2020.csv", "")

    n = len(df)
    per_file_rows.append({"ticker": ticker, "n_rows": n})

    # count missing by column (restrict to columns present)
    cols_here = [c for c in key_cols if c in df.columns]
    miss_counts = df[cols_here].isna().sum().to_dict()

    # store long-form missing counts and percents
    for c in cols_here:
        n_miss = int(miss_counts[c])
        pct = (n_miss / n * 100.0) if n > 0 else np.nan
        per_file_missing.append({
            "ticker": ticker, "column": c,
            "n_missing": n_miss, "pct_missing": pct
        })

    # rows with ANY NaN (across all columns, not just price columns)
    any_na_count = int(df.isna().any(axis=1).sum())
    any_na_pct = (any_na_count / n * 100.0) if n > 0 else np.nan
    per_file_anyrow.append({
        "ticker": ticker,
        "rows_with_any_na": any_na_count,
        "pct_rows_with_any_na": any_na_pct
    })

# === Summaries ===
rows_tbl   = pd.DataFrame(per_file_rows).sort_values("ticker").reset_index(drop=True)
missing_tbl= pd.DataFrame(per_file_missing).sort_values(["ticker","column"]).reset_index(drop=True)
anyrow_tbl = pd.DataFrame(per_file_anyrow).sort_values("ticker").reset_index(drop=True)

print("=== Rows per file ===")
print(rows_tbl.head(10))
print(f"... total files: {len(rows_tbl)}")

print("\n=== Missingness by file & column (first 20 rows) ===")
print(missing_tbl.head(20))

print("\n=== Rows containing ANY NaN per file (top 20) ===")
print(anyrow_tbl.head(20))

# === Wide summary: one row per ticker, key columns as % missing (like describe in lectures) ===
wide = (missing_tbl
        .pivot(index="ticker", columns="column", values="pct_missing")
        .fillna(0.0)
        .reset_index())

print("\n=== % missing by ticker (wide table, first 10) ===")
print(wide.head(10))

# === Quick spotlight on the main target: real_close ===
if "real_close" in missing_tbl["column"].unique():
    miss_real = (missing_tbl[missing_tbl["column"]=="real_close"]
                 .sort_values("pct_missing", ascending=False)
                 .reset_index(drop=True))
    print("\n=== Tickers with most missing in real_close ===")
    print(miss_real.head(20))

# === Optional CSV exports for your report/appendix (comment out if not needed) ===
OUT_DIR = os.path.join(CLEAN_DIR, "_missingness_checks")
os.makedirs(OUT_DIR, exist_ok=True)
rows_tbl.to_csv(os.path.join(OUT_DIR, "rows_per_file.csv"), index=False)
missing_tbl.to_csv(os.path.join(OUT_DIR, "missing_by_file_and_column.csv"), index=False)
anyrow_tbl.to_csv(os.path.join(OUT_DIR, "rows_with_any_na.csv"), index=False)
wide.to_csv(os.path.join(OUT_DIR, "pct_missing_wide.csv"), index=False)

print(f"\nSaved summaries to: {OUT_DIR}")


In [ ]:
# Outlier detection (3 std deviation)

import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


CLEAN_DIR = "/home/ug/orlovsd2/csc1171/data/clean/SP500_ETF_FX_Crypto_Daily"
files = glob.glob(f"{CLEAN_DIR}/*_cleaned_2008_2020.csv")


def load_clean_csv(path):
    try:
        df = pd.read_csv(path, parse_dates=["Date"]); df = df.set_index("Date")
    except ValueError:
        df = pd.read_csv(path, index_col=0, parse_dates=[0])
    df.index = pd.to_datetime(df.index).tz_localize(None)
    return df

def safe_pct_change(s):
    r = s.pct_change()
    return r.replace([np.inf, -np.inf], np.nan)

def flag_outliers_3sigma(series):
    """Return a DataFrame with mean, std, z, and boolean mask |z|>3."""
    s = pd.to_numeric(series, errors="coerce")
    mu = s.mean()
    sd = s.std(ddof=1)  # sample std (lecture default)
    z = (s - mu) / sd if sd not in (0, np.nan, None) and not np.isclose(sd, 0.0) else pd.Series(np.nan, index=s.index)
    mask = z.abs() > 3
    return mu, sd, z, mask

per_ticker_summary = []   # one row per ticker per variable
all_outlier_rows = []     # concatenated outlier rows for inspection

# choose variables to check (you can add/remove to match exactly your lecture scope)
CHECK_VARS = ["Return", "real_close", "volume"]

for path in files:
    df = load_clean_csv(path)
    ticker = os.path.basename(path).replace("_cleaned_2008_2020.csv", "")
    if df.empty:
        continue

    # derive daily return from real_close (as in previous cells)
    if "real_close" in df.columns:
        df["Return"] = safe_pct_change(df["real_close"])

    out_rows_this_ticker = []

    for var in CHECK_VARS:
        if var not in df.columns:
            continue

        mu, sd, z, mask = flag_outliers_3sigma(df[var])

        # per-ticker summary (counts & percents)
        n = df[var].dropna().shape[0]
        n_out = int(mask.sum()) if mask.notna().any() else 0
        pct_out = (n_out / n * 100.0) if n > 0 else np.nan

        per_ticker_summary.append({
            "ticker": ticker,
            "variable": var,
            "mean": float(mu) if pd.notna(mu) else np.nan,
            "std": float(sd) if pd.notna(sd) else np.nan,
            "n_obs": int(n),
            "n_outliers_3sigma": n_out,
            "pct_outliers_3sigma": pct_out
        })

        # collect outlier rows (with the variable, its z-score, and other context)
        if n_out > 0:
            tmp = df.loc[mask].copy()
            tmp["__variable"] = var
            tmp["__zscore"] = z[mask]
            tmp["__ticker"] = ticker
            # keep only a compact set of columns + the variable flagged
            keep_cols = [c for c in ["real_close", "close", "adj_close", "open", "high", "low", "volume", var] if c in tmp.columns]
            keep_cols = list(dict.fromkeys(keep_cols))  # de-duplicate while preserving order
            tmp = tmp[keep_cols + ["__variable", "__zscore", "__ticker"]]

            out_rows_this_ticker.append(tmp)

    # print per-ticker outliers (first 10) instead of saving
    if out_rows_this_ticker:
        out_df = pd.concat(out_rows_this_ticker).sort_index()
        print(f"\n=== {ticker}: first 10 outliers across variables ===")
        print(out_df.head(10))
        # still keep for overall combined preview
        all_outlier_rows.append(out_df.assign(__file=ticker))


# write summaries
summary_df = pd.DataFrame(per_ticker_summary).sort_values(["variable", "ticker"]).reset_index(drop=True)
print("\n=== Outlier summary by ticker & variable (top 30 rows) ===")
print(summary_df.head(30))


if all_outlier_rows:
    combined_outliers = pd.concat(all_outlier_rows).sort_index()
    print("\n=== Combined outliers (first 50 rows) ===")
    print(combined_outliers.head(50))
    print(f"\nTotal outlier rows: {len(combined_outliers)}")
else:
    print("No outliers detected at ±3σ for the checked variables (or all series have near-zero std).")



In [ ]:
# Data Transformation (We already did data normalisation), we need to do discretisation

import os, glob
import numpy as np
import pandas as pd

# controls
CLEAN_DIR = "/home/ug/orlovsd2/csc1171/data/clean/SP500_ETF_FX_Crypto_Daily"
files = glob.glob(f"{CLEAN_DIR}/*_cleaned_2008_2020.csv")

TARGET_VARS = ["Return", "real_close", "volume"]

BIN_SIZE    = 50                # number of observations per bin (lecture used 5 on a 150 sample)
pd.set_option("display.max_rows", 1000)   # increase if needed
pd.set_option("display.width", 200)
PRINT_TICKERS    = -1    # how many tickers to print (set -1 for all)
PRINT_BINS       = -1     # how many bins to show per ticker (set -1 for all)
ROWS_PER_TICKER  = 20     # how many outlier/data rows to show per ticker if you print 
ATTACH_COLUMN = True            # add a discretised column back to each df (not saved; just demo)

def load_clean_csv(path):
    try:
        df = pd.read_csv(path, parse_dates=["Date"]); df = df.set_index("Date")
    except ValueError:
        df = pd.read_csv(path, index_col=0, parse_dates=[0])
    df.index = pd.to_datetime(df.index).tz_localize(None)
    return df

def safe_pct_change(s):
    r = s.pct_change()
    return r.replace([np.inf, -np.inf], np.nan)

def discretise_by_fixed_bin_size(s, bin_size):
    """
    Lecture-style discretisation:
      1) sort values
      2) split into contiguous bins of equal size
      3) compute Bin Mean, Boundaries, Median
      4) map each original index to its bin id
    Returns:
      bin_table: DataFrame with one row per bin (mean/bounds/median/count)
      labels:    Series aligned to original index with integer bin ids [0..B-1]
    """
    s = pd.to_numeric(s, errors="coerce").dropna()
    if s.empty:
        return pd.DataFrame(), pd.Series(index=s.index, dtype="Int64")

    # sort values (like b = np.sort(b) in the lecture)
    sorted_vals = s.sort_values()
    n = len(sorted_vals)
    n_bins = max(1, n // bin_size)  # floor
    if n_bins == 1:
        # everything in one bin; still return lecture-style outputs
        v = sorted_vals.values
        row = {
            "bin_id": 0,
            "lower_bound": float(v[0]),
            "upper_bound": float(v[-1]),
            "bin_mean": float(np.mean(v)),
            "bin_median": float(np.median(v)),
            "count": int(len(v))
        }
        bin_table = pd.DataFrame([row])
        labels = pd.Series(0, index=s.index, dtype=int)
        return bin_table, labels

    # chunk indices
    edges = list(range(0, n, bin_size))
    if edges[-1] != n:  # ensure last edge at n
        edges.append(n)

    rows = []
    # map original index to bin via the *rank order* after sorting
    # build a series of bin ids for sorted index first
    label_sorted = pd.Series(index=sorted_vals.index, dtype=int)

    for b in range(len(edges) - 1):
        i0, i1 = edges[b], edges[b+1]
        block = sorted_vals.iloc[i0:i1]
        v = block.values

        # lecture stats
        lower = float(v[0])
        upper = float(v[-1])
        mean  = float(np.mean(v))
        med   = float(np.median(v))

        rows.append({
            "bin_id": b,
            "lower_bound": lower,
            "upper_bound": upper,
            "bin_mean": mean,
            "bin_median": med,
            "count": int(len(v))
        })

        # assign label b to these rows
        label_sorted.loc[block.index] = b

    bin_table = pd.DataFrame(rows)

    # reorder labels back to original time index
    labels = label_sorted.reindex(s.index).astype(int)

    return bin_table, labels

preview_left = PRINT_TICKERS
for path in files:
    df = load_clean_csv(path)
    ticker = os.path.basename(path).replace("_cleaned_2008_2020.csv", "")
    if df.empty:
        continue

    # derive Return for this discretisation step (matches earlier cells)
    if "real_close" in df.columns and "Return" not in df.columns:
        df["Return"] = safe_pct_change(df["real_close"])

    for var in TARGET_VARS:
        if var not in df.columns:
            continue

        bin_table, labels = discretise_by_fixed_bin_size(df[var], BIN_SIZE)

        # attach discretised labels back (if requested)
        if ATTACH_COLUMN and not labels.empty:
            df[f"{var}_bin"] = labels.astype("Int64") 


        if PRINT_TICKERS == -1 or preview_left > 0:
            print(f"\n=== {ticker} • {var} • BIN_SIZE={BIN_SIZE} ===")
            print("Bin Mean / Median / Boundaries:")
            if PRINT_BINS == -1:
                print(bin_table)             # print ALL bins
            else:
                print(bin_table.head(PRINT_BINS))
            # optional: show more label rows
            print("\nDiscretised labels sample (date → bin_id):")
            print(df[[var, f"{var}_bin"]].dropna().head(ROWS_PER_TICKER))
            if PRINT_TICKERS != -1:
                preview_left -= 1


# Note:
# If you want equal-*width* bins instead (not equal-frequency), swap logic to:
# pd.cut(s, bins=n_bins, right=True, include_lowest=True) and compute mean/median per category.

